This agent uses an optimal assignment algorithm to select targets
for each ship. The value of targets is as described in
https://www.kaggle.com/solverworld/optimal-mining-with-carried-halite.
An earlier version of this agorithm is discussed in notebook
kaggle.com/solverworld/optimal-mining.

# Create Environment

In [ ]:
from kaggle_environments import evaluate, make
env = make("halite", debug=True)
env.render()

# Create Agent

In [ ]:
%%writefile submission.py
import time
import copy
import sys
import math
import collections
import pprint
import numpy as np
import scipy.optimize
import scipy.ndimage
from kaggle_environments.envs.halite.helpers import *
import kaggle_environments
import random

'''
Initialization code can run when the file is loaded.
The first call to agent() is allowed 30 sec
The agent function is the last function in the file
(does not matter its name).
Agent must run in less than 6 sec.
Syntax errors when running the file do not cause any message,
but cause your agent to do nothing
'''

SHOW_LOGS = True
CONFIG_MAX_SHIPS = 50
CONFIG_MAX_SHIPYARDS = 6
ATTACK_MIN_DISTANCE_SHIP = 5
ATTACK_SEPARATION_SHPIPYARDS = 10
ATTACK_MIN_HALITE_CELL = 50
HALITE_MIN = 50
AVOID_POSITIONS_ENEMY_SHIP_SAME_HALITE = True

# Game phases
STEP_START_HUNTING = 100
STEP_START_GAIN_REWARD = 260
STEP_START_CLOSE_DOWN = 390

# MIN_N_SHIPS_TO_ATTACK_SHIPYARD = 10

all_actions = [
    ShipAction.NORTH,
    ShipAction.EAST,
    ShipAction.SOUTH,
    ShipAction.WEST]

all_dirs = [
    Point(0, 1),
    Point(1, 0),
    Point(0, -1),
    Point(-1, 0)]

start = None
size = None
ship_target = {}
me = None
did_init = False

# quiet True if you want quiet output.
quiet = False

C = None


class Obj:
    pass

# The Obj class will hold global data for this turn, updating as we set actions.
# E.g. number of ships, amount of halite...
turn = Obj()

# turns_optimal is a matrix that defines the optimal configuration to
# mine halite. It is of the shape turns_optimal[CHratio, round_trip_travel].
# Ie, columns is the number of cells to travel from the ship to the halte 
# + the shipyard. The rows indicate the ratio of halite carried by the ship (C)
# and the halite in the position intented to mine.
# Each cell indicates the number of turns the ship should spend there.
# See notebook on optimal mining kaggle.com/solverworld/optimal-mining
turns_optimal = np.array(
    [[0, 2, 3, 4, 4, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 8],
     [0, 1, 2, 3, 3, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 7, 7, 7],
     [0, 0, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 7],
     [0, 0, 1, 2, 2, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6],
     [0, 0, 0, 1, 2, 2, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6],
     [0, 0, 0, 0, 0, 1, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5],
     [0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


def print_enemy_ships(board):
    print('\nEnemy Ships')
    for ship in board.ships.values():
        if ship.player_id != me.id:
            print('Ship: {0}. Position: {1}. Next action: {2}. Halite carrying: {3}.'
                  .format(ship.id, ship.position, ship.halite))


def print_actions(board):
    print('\nShip Actions')
    for ship in me.ships:
        print('Ship: {0}. Position: {1}. Next action: {2}. Halite carrying: {3}.'
              .format(ship.id, ship.position, ship.next_action, ship.halite))
        
    print('\nShipyard Actions')
    for sy in me.shipyards:
        print('Shipyard: {0}. Position: {1}. Next action: {2}.'
              .format(sy.id, sy.position, sy.next_action))


def print_none(*args):
    pass


def compute_max_ships(step):
    # Tune number of max ships based on the turn.
    if step < 200:
        return CONFIG_MAX_SHIPS
    elif step < 300:
        return CONFIG_MAX_SHIPS # 200-300
    elif step < 350:
        return CONFIG_MAX_SHIPS - 30 # 300-350
    elif step < 380:
        return CONFIG_MAX_SHIPS - 40 # 350-380
    elif step < 390:
        return CONFIG_MAX_SHIPS - 45 # 380-390
    elif step < 395:
        return CONFIG_MAX_SHIPS - 49 # 390-395
    else:
        return CONFIG_MAX_SHIPS - 49  # 395-400
    
    
def compute_winning_position(board):
    ships_enemy_players = {}
    total_halite_enemy_players = {}

    # Iterate over all players:
    for player in board.players:
        if player != me.id:
            total_halite_enemy_ships_carrying = 0
            for ship in board.players[player].ships:
                total_halite_enemy_ships_carrying += ship.halite
            ships_enemy_players.update({player: len(board.players[player].ships)})
            total_halite_enemy_players.update({player: board.players[player].halite + total_halite_enemy_ships_carrying})

    # Enemy with more ships:
    player_more_ships = max(ships_enemy_players, key=ships_enemy_players.get)
    # Enemy with more halite:
    player_more_halite = max(total_halite_enemy_players, key=total_halite_enemy_players.get)
    
    # print("player_more_ships:", player_more_ships)
    # print("player_more_halite", player_more_halite)
    
    return player_more_ships, player_more_halite
    

def set_turn_data(board):
    # Initialize the global turn data for this turn.
    turn.num_ships = len(me.ships)
    turn.num_shipyards = len(me.shipyards)
    turn.max_ships = compute_max_ships(board.step)
    turn.total_halite = me.halite
    
    # This is the matrix of halite in cells
    turn.halite_matrix = np.reshape(
        board.observation['halite'],
        (board.configuration.size, board.configuration.size))
    
    player_more_ships, player_more_halite = compute_winning_position(board)
        
    # Compute enemy presence and enemy halite matrices.
    turn.EP, turn.EH, turn.ES, turn.ES2 = gen_enemy_halite_matrix(board, player_more_ships, player_more_halite)
    
    # DIctionary filled in by ship_id as a ship takes up a square.
    turn.taken = {}
    
    # Last turn of the game.
    turn.last_episode = (board.step == (board.configuration.episode_steps - 2))


def init(obs, config):
    # This is only called on first call to agent()
    # It does the initialization.
    
    global size
    global print
    
    # If SHOW_LOGS, return prints (not quiet).
    if SHOW_LOGS:      
        pass
    # Otherwise, return quiet output.
    else:
        print = print_none
        pprint.pprint = print_none
        
    size = config.size


def limit(x, a, b):
    if x < a:
        return a
    if x > b:
        return b
    return x


def num_turns_to_mine(C, H, rt_travel):
    # How many turns should we plan on mining.
    # C: carried halite
    # H: halite in square/cell
    # rt_travel: steps to square and back to shipyard
    if C == 0:
        ch = 0
    # If halite 0, we will always return 0 as value.
    elif H == 0:
        ch = turns_optimal.shape[0] - 1
    else:
        ch = int(math.log(C / H) * 2.5 + 5.5)
        ch = limit(ch, 0, turns_optimal.shape[0] - 1)
    rt_travel = int(limit(rt_travel, 0, turns_optimal.shape[1] - 1))
    
    # Based on the ratio C/H and rt_travel we return the turns we should stay 
    # in that cell mining.
    return turns_optimal[ch, rt_travel]



def halite_per_turn(board, carrying, halite, distance_to_pt, distance_to_shipyard, max_separation, halite_min, min_mine=1):
    # Compute the reward/value of going to a certain cell to mine.
    # travel: number of turns it will take us to travel to the cell.
    
    travel = distance_to_pt + distance_to_shipyard
    
    if halite < halite_min:
        reward = 0
        
    else:
        turns = num_turns_to_mine(carrying, halite, travel)

        # we can hard code a minimal number to be mining (min_mine).
        # Not doing it doesn't cause any problem.
        if turns < min_mine:
            turns = min_mine

        # Compute the total amount mined we expect to have after mining the amount of turns chose.
        mined = (1 - .75**turns) * halite + carrying

        if mined < 0:
            mined = 0

        if board.step > STEP_START_GAIN_REWARD:
            if distance_to_shipyard >= 12:
                mined = 0
                
        else:            
            if distance_to_shipyard >= 8:
                mined = 0
            

        # We return reward/value for going to that cell to mine, and turns mining (no travel included).
        reward = mined / (travel + turns)

    return reward  # usually less than 300


def move(pos, action):
    # Return new Position from pos when action is applied.

    ret = None
    
    if action == ShipAction.NORTH:
        ret = pos + Point(0, 1)
    if action == ShipAction.SOUTH:
        ret = pos + Point(0, -1)
    if action == ShipAction.EAST:
        ret = pos + Point(1, 0)
    if action == ShipAction.WEST:
        ret = pos + Point(-1, 0)
    if ret is None:
        ret = pos
        
    # print('move pos {} {} => {}'.format(pos,action,ret))
    return ret % size


def dirs_to(p1, p2, size=21):
    # Obtin the actions we should take to go from Point p1 to Point p2 
    # using shortest direction by wraparound.
    # p1: Point 1 (origin).
    # p2: Point 2 (final).
    # size:  size of board.
    # returns: list of directions, tuple (deltaX,deltaY)
    # The list is of length 1 or 2 giving possible directions to go, e.g.
    # to go North-East, it would return [ShipAction.NORTH, ShipAction.EAST],
    # because you could use either of those first to go North-East.
    # [None] is returned if p1==p2 and there is no need to move at all.
    
    deltaX, deltaY = p2 - p1
    
    if abs(deltaX) > size / 2:
        # we wrap around.
        if deltaX < 0:
            deltaX += size
        elif deltaX > 0:
            deltaX -= size
            
    if abs(deltaY) > size / 2:
        # we wrap around.
        if deltaY < 0:
            deltaY += size
        elif deltaY > 0:
            deltaY -= size
            
    # The delta is (deltaX,deltaY).
    ret = []
    if deltaX > 0:
        ret.append(ShipAction.EAST)
    if deltaX < 0:
        ret.append(ShipAction.WEST)
    if deltaY > 0:
        ret.append(ShipAction.NORTH)
    if deltaY < 0:
        ret.append(ShipAction.SOUTH)
    if len(ret) == 0:
        # Do not need to move at all.
        ret = [None]  
        
    return ret, (deltaX, deltaY)


def shipyard_actions(board):
    # Spawn a ship as long as there is no ship already moved to this shipyard.
    
    option = 1
    
    # OPTION 1: Spawn ships once we have 500
    # OPTION 2: Spawn ships once we have 1000 (combined with defense shipyards spawn ships)
    
    if option == 1:
        # Do not create ships until we have created the second shipyard.
        # Remember we ask 1000 halite to create the shipyard.
        if turn.num_shipyards < 2:
            turn.max_ships = 9

        elif turn.num_shipyards < 3:
            turn.max_ships = 18

        elif turn.num_shipyards < 4:
            turn.max_ships = 21

        elif turn.num_shipyards < 5:
            turn.max_ships = 27

        elif turn.num_shipyards < 6:
            turn.max_ships = 31

        max_ships = compute_max_ships(board.step)

        if max_ships < turn.max_ships:
            turn.max_ships = max_ships
        
        """create_ship = True
        
        if turn.num_shipyards < 2 and len(me.ships) == 9:
            create_ship = False
            
        elif turn.num_shipyards < 3 and board.step >= STEP_START_HUNTING and len(me.ships) > 12:
            create_ship = False

        elif turn.num_shipyards < 4 and board.step >= STEP_START_HUNTING + 50 and len(me.ships) > 16:
            create_ship = False

        elif turn.num_shipyards < 5 and turn.num_ships > 29:
            create_ship = False
            
        else:
            create_ship = True"""

        if turn.num_ships < turn.max_ships and turn.total_halite >= 500:
            for sy in me.shipyards:
                d = nearest_enemy_ship(board, sy.position)
                # we prioritize spawning from shipyard that can be attacked.
                if d < 3:
                    if turn.num_ships < turn.max_ships:
                        if (turn.total_halite >= 500 and sy.position not in turn.taken):
                            # Spawn one.
                            sy.next_action = ShipyardAction.SPAWN
                            turn.taken[sy.position] = 1
                            turn.num_ships += 1
                            turn.total_halite -= 500

            for sy in me.shipyards:
                d = nearest_enemy_ship(board, sy.position)
                # we prioritize spawning from shipyard that can be attacked.
                if d < 4:
                    if turn.num_ships < turn.max_ships:
                        if (turn.total_halite >= 500 and sy.position not in turn.taken):
                            # Spawn one.
                            sy.next_action = ShipyardAction.SPAWN
                            turn.taken[sy.position] = 1
                            turn.num_ships += 1
                            turn.total_halite -= 500

            for sy in me.shipyards:
                if turn.num_ships < turn.max_ships:
                    if (turn.total_halite >= 500 and sy.position not in turn.taken):
                        # Spawn one.
                        sy.next_action = ShipyardAction.SPAWN
                        turn.taken[sy.position] = 1
                        turn.num_ships += 1
                        turn.total_halite -= 500
                        
                        
    elif option == 2:
        if turn.num_ships < turn.max_ships and turn.total_halite >= 500:
            if len(me.ships) < 9:
                for sy in me.shipyards:
                # Spawn a ship as long as there is no ship already moved to this shipyard.
                # If we have 9 or more ships, we want to keep some halite in reserve to defend shipyards.
                    if turn.total_halite >= 500 and sy.position not in turn.taken:
                        # Spawn one.
                        sy.next_action = ShipyardAction.SPAWN
                        turn.taken[sy.position] = 1
                        turn.num_ships += 1
                        turn.total_halite -= 500
                    
            elif turn.total_halite >= 1000:
                for sy in me.shipyards:
                    d = nearest_enemy_ship(board, sy.position)
                    # we prioritize spawning from shipyard that can be attacked.
                    if d < 3:
                        if turn.num_ships < turn.max_ships:
                            # If we have 9 or more ships, we want to keep some halite in reserve to defend shipyards.
                            # if (len(me.ships) < 9 and turn.total_halite >= 500 and sy.position not in turn.taken) \
                                # or (turn.total_halite >= 1000 and sy.position not in turn.taken):
                            if (turn.total_halite >= 1000 and sy.position not in turn.taken):
                                # Spawn one.
                                sy.next_action = ShipyardAction.SPAWN
                                turn.taken[sy.position] = 1
                                turn.num_ships += 1
                                turn.total_halite -= 500

                for sy in me.shipyards:
                    d = nearest_enemy_ship(board, sy.position)
                    # we prioritize spawning from shipyard that can be attacked.
                    if d < 4:
                        if turn.num_ships < turn.max_ships:
                            # If we have 9 or more ships, we want to keep some halite in reserve to defend shipyards.
                            # if (len(me.ships) < 9 and turn.total_halite >= 500 and sy.position not in turn.taken) \
                                # or (turn.total_halite >= 1000 and sy.position not in turn.taken):
                            if (turn.total_halite >= 1000 and sy.position not in turn.taken):
                                # Spawn one.
                                sy.next_action = ShipyardAction.SPAWN
                                turn.taken[sy.position] = 1
                                turn.num_ships += 1
                                turn.total_halite -= 500

                for sy in me.shipyards:
                    if turn.num_ships < turn.max_ships:
                        # If we have 9 or more ships, we want to keep some halite in reserve to defend shipyards.
                        # if (len(me.ships) < 9 and turn.total_halite >= 500 and sy.position not in turn.taken) \
                        #     or (turn.total_halite >= 1000 and sy.position not in turn.taken):
                        if (turn.total_halite >= 1000 and sy.position not in turn.taken):
                            # Spawn one.
                            sy.next_action = ShipyardAction.SPAWN
                            turn.taken[sy.position] = 1
                            turn.num_ships += 1
                            turn.total_halite -= 500
        

def gen_enemy_halite_matrix(board, player_more_ships, player_more_halite):
    # Generate matrix of enemy positions:
    # EP: presence of enemy ship.
    # EH: amount of halite in enemy ships.
    # ES: presence of enemy shipyards.
    # ES2: presense of enemy shipyards if conditons met.
    # EA: presence of enemy attack.
    
    EP = np.zeros((size, size))
    EH = np.zeros((size, size))
    ES = np.zeros((size, size))
    ES2 = np.zeros((size, size))
    # EA = np.zeros((size, size)) 
    
    # Iterate over my ships.
    # for ship in me.ships:
    #     x = ship.position.x
    #     y = ship.position.y
    #     # Check if any enemy ship at d=1.
    #     for id, enemy_ship in board.ships.items():
    #         if enemy_ship.player_id != me.id:
    #             if dist(enemy_ship.position, ship.position) == 1:
    #                 EA[y, x] = 1    
    
    # Iterate over the ships of the board.
    for id, ship in board.ships.items():
        # If enemy's ship, save halite and position of it in matrices.
        if ship.player_id != me.id:
            EH[ship.position.y, ship.position.x] = ship.halite
            EP[ship.position.y, ship.position.x] = 1
            
    # Iterate over the shipyards of the board.
    for id, sy in board.shipyards.items():
        # If enemy's shipyard, store the position in a matrix.
        if sy.player_id != me.id:
            ES[sy.position.y, sy.position.x] = 1
            # If mets conditions, store positon in second matrix to avoid attacking.                
            if sy.player_id != player_more_ships and sy.player_id != player_more_halite:
                ES2[sy.position.y, sy.position.x] = 1
            
    return EP, EH, ES, ES2


def dist(a, b):
    # Manhattan distance of the Point difference a to b, considering wrap around.
    action, step = dirs_to(a, b, size=21)
    return abs(step[0]) + abs(step[1])


def nearest_shipyard(pos):
    # Return distance and position of nearest shipyard to position.
    # Return 100, None if no shipyards.
    mn = 100
    best_pos = None
    for sy in me.shipyards:
        d = dist(pos, sy.position)
        if d < mn:
            mn = d
            best_pos = sy.position
            
    return mn, best_pos


def nearest_enemy_shipyard(board, pos):
    mn = 100
    for id, shipyard in board.shipyards.items():
        if shipyard.player_id != me.id:
            d = dist(pos, shipyard.position)
            if d < mn:
                mn = d
            
    return mn


def nearest_enemy_ship(board, pos):
    mn = 100
    for id, ship in board.ships.items():
        if ship.player_id != me.id:
            d = dist(pos, ship.position)
            if d < mn:
                mn = d
            
    return mn


def nearest_ally_ship(board, pos):
    mn = 100
    best_pos = None
    for id, ship in board.ships.items():
        if ship.player_id == me.id:
            d = dist(pos, ship.position)
            if d < mn:
                mn = d
                best_ship = ship
            
    return mn, best_ship


def assign_targets(board, ships, player_more_ships, player_more_halite):
    # Assign the ships to a cell, ie set ship_target[ship_id] to a Position.
    # Assign rewards to cells based on which cells maximize halite per step. 
    # Return to deposit the halite into a shipyard if that is optimal.
    # Make a list of pts containing cells we care about, this will be our columns of matrix C.
    # The rows of matrix C are the ships we currently have.
    # Computes global dict ship_target with shipid->Position for its target.
    # Global ship targets should already exist.
    old_target = copy.copy(ship_target)
    ship_target.clear()
    
    HALITE_MIN = 50
    
    if board.step < STEP_START_HUNTING:
        HALITE_MIN = 30
    
    if board.step > STEP_START_GAIN_REWARD:
        HALITE_MIN = 30
    
    # If we don't have ships, returns nothing.
    if len(ships) == 0:
        return
    
    # We only want to store the points we really need.
    # If we store all cells of the board, we can have timeouts (>6s per turn).
    pts1 = []
    pts2 = []
    pts3 = []
    
    for pt,c in board.cells.items():
        assert isinstance(pt, Point)
        if c.halite > HALITE_MIN:
            pts1.append(pt)
            
    # Iterate over the shipyards of the board.
    for id, sy in board.shipyards.items():
        pts2.append(sy.position)
        
    # Iterate over the ships of the board.
    for id, ship in board.ships.items():
        pts3.append(ship.position)
        
    pts = pts1 + pts2 + pts3
    
    # Remove duplicates of the list.
    pts = list(dict.fromkeys(pts))
    
    
    """
    # If we want to store all cells of the board
    pts = []
    
    # We consider all the points of the board.
    for pt, c in board.cells.items():
        assert isinstance(pt, Point)
        pts.append(pt)
    """
            
    # C is the matrix that will assign values for each ship point combination C[ship,pt].
    C = np.zeros((len(ships), len(pts)))
    
    # We proceed to fill the matrix C.
    for i, ship in enumerate(ships):
        for j, pt in enumerate(pts):
            
            # d1: distance from ship to point pt.
            # d2: distance from point pt to nearest shipyard.
            d1 = dist(ship.position, pt)
            d2, shipyard_position = nearest_shipyard(pt)
            
            # If we dont have any shipyard, assing d2 = 1.
            if shipyard_position is None:
                d2 = 1
                
            # Amount of halite of my ship.
            my_halite = ship.halite
            
            # Dynamic HALITE_MIN
            HALITE_MIN = 50
            
            if board.step < STEP_START_HUNTING:
                HALITE_MIN = 50
            
            if len(me.shipyards) == 3 and len(me.ships) > 12:
            #or board.step >= STEP_START_HUNTING:
                HALITE_MIN = 30
                
            if len(me.shipyards) == 4 and len(me.ships) > 18:
                HALITE_MIN = 90
             
            if board.step >= STEP_START_HUNTING + 100 and len(me.ships) > 12:
                HALITE_MIN = 150
            
            if board.step >= STEP_START_HUNTING + 150 and len(me.ships) > 12:
                HALITE_MIN = 200
                
            if board.step > STEP_START_GAIN_REWARD:
                HALITE_MIN = 30              
                
            
            """
            if board.step < 30:
                if d2 < 6:
                    HALITE_MIN = 400
                    
            if board.step < 50:
                if d2 < 3:
                    HALITE_MIN = 100
                    
            if len(me.ships) > 20:
                if d2 < 3:
                    HALITE_MIN = 300
                    
            if len(me.ships) < 12:
                HALITE_MIN = 50
                                    
            if board.step > 350:
                if d2 < 3:
                    HALITE_MIN = 30
            """
            
            # Iterate overall the cells to compute a value given the halite we can mine.
            # We assign rewards to cells based on which cells maximize halite per step.
            v = halite_per_turn(board=board,
                                carrying=my_halite,
                                halite=board.cells[pt].halite,
                                distance_to_pt=d1,
                                distance_to_shipyard=d2,
                                max_separation=ATTACK_SEPARATION_SHPIPYARDS,
                                halite_min=HALITE_MIN,
                                min_mine=1) 
            # usually less than 300
            
            # If someone else's ship on the cell.
            if board.cells[pt].ship:
                # If it is enemy's ship see how much halite they have.
                if board.cells[pt].ship.player_id != me.id:
                    enemy_halite = board.cells[pt].ship.halite
                    # If he has less halite, we don't want to go there.
                    if enemy_halite < my_halite:
                        v = -10000
                    else:
                    # Otherwise, we attack or ignore it depending on the reward.
                    # We follow the ship as long as we don't separate too much
                    # from our shipyards.
                    # Also it is important to notice that we should only attack to
                    # ships where the halite of the cell is above a certain amount.
                    # Otherwise, the enemy ship will not stay in that cell in the next turn.
                    # Before we arrive, he will have moved.
                        if (board.step <= STEP_START_HUNTING and len(me.ships) >= 20) or \
                                (board.step > STEP_START_HUNTING and board.step < STEP_START_GAIN_REWARD and len(me.ships) >= 12):
                            if d1 < ATTACK_MIN_DISTANCE_SHIP and d2 < ATTACK_SEPARATION_SHPIPYARDS:
                                # and board.cells[pt].halite >= ATTACK_MIN_HALITE_CELL:
                                if my_halite != 0:
                                    v += enemy_halite + 2000 / (d1 + 1)
                                else:
                                    v += enemy_halite + 2000 / (d1 + 1)
                        """            
                        elif board.step > STEP_START_GAIN_REWARD:
                            if d1 < ATTACK_MIN_DISTANCE_SHIP and d2 < ATTACK_SEPARATION_SHPIPYARDS:
                                # and board.cells[pt].halite >= ATTACK_MIN_HALITE_CELL:
                                if my_halite != 0:
                                    v += enemy_halite / (d1 + 1)
                                else:
                                    v += enemy_halite + 10 / (d1 + 1)"""
                

                # If it is one of my ships:
                if board.cells[pt].ship.player_id == me.id:
                    # ally ship but not the onw we are iterating on.
                    if ship != board.cells[pt].ship:
                        #ally_ship_halite = board.cells[pt].ship.halite
                        # We remove the score of the cell if we already have a ship there. 
                        # Our ally ship may be already mining or going to mine that cell
                        # so we do not want to go there.
                        v = 0

                    """
                    # If he has more halite, the game is close to end, and
                    # the ally_ship is closer to a shipyard than I am,
                    # and the distance between us is 1.
                    # Then we want to go there.
                    # d1 is the distance of my ships and the ally_ship
                    # d2 is the distance of the ally_ship and the nearest shipyard.
                    # d3 is the distance of my ship and the nearest shipyard.
                    d3, shipyard_position = nearest_shipyard(ship.position)
                    if ally_ship_halite > my_halite and board.step > STEP_CLOSE_TO_END and d2 < d3 and d1 == 1:
                        v += 2000    
                        """
                
            # Iterate over the cells where we have a shipyard.
            if board.cells[pt].shipyard:
                # If it is one of my shipyards.
                if board.cells[pt].shipyard.player_id == me.id:
                    # If the shipyard is separated from the ship more than 0 distance, we assign
                    # a score based on the distance and my halite.
                    if d1 > 0:
                        v = my_halite / d1 # usually less than 300
                    else:
                        # If the distance is 0, means that we are in a shipyard, which means that 
                        # we have already deposit, so we don't want to stay there.
                        v = -2
                
                # If it is enemy's shipyard.
                if board.cells[pt].shipyard.player_id != me.id:
                    # Attack the player who is winning.
                    if (my_halite == 0 and ( \
                        board.cells[pt].shipyard.player_id == player_more_ships \
                        or board.cells[pt].shipyard.player_id == player_more_halite) \
                    ) \
                    or (my_halite < 100 and d1 < 3 and ( \
                        board.cells[pt].shipyard.player_id == player_more_ships \
                        or board.cells[pt].shipyard.player_id == player_more_halite) \
                        and (board.players[board.cells[pt].shipyard.player_id].halite < 500)
                    ):
                        # Only add reward if I have more than 25 ships.
                        # = and not += because if there is a ship will mess the reward.
                        if len(me.ships) > 25:
                            # d1: distance from ship to point pt.
                            v = (500 - my_halite) / d1
                        # If the board step is > 300, then I only need 5 ships.
                        if (board.step > 300 and len(me.ships) > 15 \
                                and board.cells[pt].shipyard.player_id == player_more_halite):
                            v = 10000
                            
                        
            # We assign the reward/value of each cell of the board.
            C[i, j] = v
            
    # print('C is {}'.format(C.shape))
    # print(C)
    
    # Compute the optimal assignment:
    row, col = scipy.optimize.linear_sum_assignment(C, maximize=True)
    # After this, ship row[i] is assigned to target col[j].

    for r, c in zip(row, col):
        ship_target[ships[r].id] = pts[c]
    
    # Print out results.
    print('\nShip Targets')
    for id, t in ship_target.items():
        ta = 'No'
        if board.ships[id].position == t:
            st = 'MINE CURRENT CELL'
        elif len(me.shipyards) > 0 and t == me.shipyards[0].position:
            st = 'MOVE TO SHIPYARD'
        else:
            st = 'MOVE TO CELL TARGET'
        if id not in old_target or old_target[id] != ship_target[id]:
            ta = ' Yes'
        print('Ship: {0}. Position: ({1[0]:2},{1[1]:2}). Target position: ({2[0]:2},{2[1]:2}). Halite cell: {3:.2f}. Target action: {4}. New target?: {5}'
              .format(id, board.ships[id].position, t, board.cells[t].halite, st, ta))

    return


def make_avoidance_matrix(myship_halite):
    # Make a matrix of booleans, where True are the
    # positions we want to avoid.
    # turn.EP: enemy's position matrix
    # turn.EH: enemy's halite matrix
    # turn.ES: enemy's shipyard matrix
    # turn.ES2: enemy's shipyard matrix when conditions met.
    # turn.EA: presence of enemy attack.

    # filter used for the convolution.
    filter = np.array([[0, 1, 0], [1, 1, 1], [0, 1, 0]])
    
    # If enemy's ship halite is lower than my ship halite, avoid that position.
    
    if AVOID_POSITIONS_ENEMY_SHIP_SAME_HALITE:
        bad_ship = np.logical_and(turn.EH <= myship_halite, turn.EP)
    else:
        bad_ship = np.logical_and(turn.EH < myship_halite, turn.EP)
    
    """
    bad_ship = np.logical_and(turn.EH <= myship_halite, turn.EP)
    if len(me.ships) >= 25:
        bad_ship = np.logical_and(turn.EH < myship_halite, turn.EP)
    """


    # The convolution avoids the 4 positions where the enemy ship can move.
    avoid = scipy.ndimage.convolve(bad_ship, filter, mode='wrap', cval=0.0)
    # avoid = np.logical_or(avoid, turn.ES)
    
    return avoid


def make_attack_matrix(myship_halite):
    # Make a matrix of booleans, where True are the positions
    # we would want to move to attack an enemy ship.
    # turn.EP: enemy position matrix
    # turn.EH: enemy halite matrix
    
    attack = np.logical_and(turn.EH > myship_halite, turn.EP)
    
    # print('attack',attack)
    return attack


def make_attack_shipyard_matrix(myship_halite):
    # Make a matrix of booleans, where True are the positions
    # we would want to move to attack an enemy shipyard.
    # turn.ES: enemy's shipyard matrix
    
    attack = np.logical_and(myship_halite < 100, turn.ES)
    
    # print('attack',attack)
    return attack


def get_max_halite_ship(board, avoid_danger=True):
    # Return my ship carrying max halite, or None if no ships
    # NOTE: creating avoid matrix again!
    
    mx = -1
    the_ship = None
    
    # Iterate over my ships.
    for ship in me.ships:
        x = ship.position.x
        y = ship.position.y
        avoid = make_avoidance_matrix(ship.halite)
        
        # If ship halite is higher than mx and the ship is not in danger
        # we update mx, and we select ship.
        if ship.halite > mx and (not avoid_danger or not avoid[y, x]):
            mx = ship.halite
            the_ship = ship
            
    return the_ship


def remove_dups(p):
    # Remove duplicates from a list without changing order.
    # Not efficient for long lists.
    ret = []
    for x in p:
        if x not in ret:
            ret.append(x)
    return ret


def matrix_lookup(matrix, pos):
    return matrix[pos.y, pos.x]


def shipyard_defense_move_ships(board):
    option = 1
        
    if option == 1:
        # If enemy ship close, select closest ally ship to defend shipyard.
        for sy in me.shipyards:
            if sy.position not in turn.taken:
                d = nearest_enemy_ship(board, sy.position)

                if d == 3:
                    d2, ship = nearest_ally_ship(board, sy.position)
                    if d2 < 3 and d2 != 0:
                        a, delta = dirs_to(ship.position, sy.position)
                        ship.next_action = a[0]  # a is a list of actions
                        turn.taken[sy.position] = 1

                if d == 2:
                    d2, ship = nearest_ally_ship(board, sy.position)
                    if d2 < 2:
                        a, delta = dirs_to(ship.position, sy.position)
                        ship.next_action = a[0]  # a is a list of actions
                        turn.taken[sy.position] = 1

                # If enemy is next to shipyard and we have a ship there,
                # we make the ship stay.
                if d == 1:
                    d2, ship = nearest_ally_ship(board, sy.position)
                    if d2 == 0:
                        ship.next_action = None
                        
    elif option == 2:
        # this option is to combine with defense shipyard spawn.
        for sy in me.shipyards:
            if sy.position not in turn.taken:
                d = nearest_enemy_ship(board, sy.position)
                """
                if d == 2:
                    d2, ship = nearest_ally_ship(board, sy.position)
                    if d2 < 2:
                        a, delta = dirs_to(ship.position, sy.position)
                        ship.next_action = a[0]  # a is a list of actions
                        turn.taken[sy.position] = 1"""

                # If enemy is next to shipyard and we have a ship there,
                # we make the ship stay.
                if d == 1:
                    d2, ship = nearest_ally_ship(board, sy.position)
                    if d2 == 0:
                        ship.next_action = None
    

def shipyard_defense_spawn_ships(board):
    # Before check if we have enough halite, otherwise we cannot save it.
    # We don't want to save after turn 350.
    if turn.total_halite >= 500 and board.step < 350 and len(me.ships) <= 50:
        # We iterate over all our shipyards to see if some of them is in danger.
        for sy in me.shipyards:
            # Check no one is moving to the shipyard this turn.
            if sy.position not in turn.taken:
                # Check danger                
                # turn.EP: enemy's position matrix
                # The convolution marks the 4 positions where the enemy ship can move.
                # Filter used for the convolution.
                filter = np.array([[0, 1, 0], [1, 1, 1], [0, 1, 0]])
                enemy_movements = scipy.ndimage.convolve(turn.EP, filter, mode='wrap', cval=0.0)
                
                x = sy.position.x
                y = sy.position.y
                
                danger = False
                if enemy_movements[y, x]:
                    danger = True
                
                if danger:
                    # Spawn a ship
                    sy.next_action = ShipyardAction.SPAWN
                    turn.taken[sy.position] = 1
                    turn.num_ships += 1
                    turn.total_halite -= 500
    
    
def ship_converts(board):
    def converts(ship, turn):
        ship.next_action = ShipAction.CONVERT
        turn.taken[ship.position] = 1
        turn.num_shipyards += 1
        turn.total_halite -= 500

    # If no shipyard, convert the ship carrying max halite unless it is in danger.
    if turn.num_shipyards == 0 and not turn.last_episode:
        ship = get_max_halite_ship(board)
        if ship is not None:
            if ship.halite + turn.total_halite >= 500:
                converts(ship, turn)
                
    # Now check the rest to see if they should convert.
    for ship in me.ships:
        
        # If the ship already has an action assigned, jump to the next ship (iteration)
        if ship.next_action:
            continue
            
        # Check if ship in danger without escape, convert if h>500
        # What this does is avoiding the enemy to take my halite and store the halite that exceeds 500.
        avoid = make_avoidance_matrix(ship.halite)
        z = [matrix_lookup(avoid, move(ship.position, a)) for a in all_actions]
        if np.all(z) and ship.halite >= 500:
            converts(ship, turn)
            print('ship id {} no escape converting'.format(ship.id))
            
        # Check if last step and > 500 halite, convert
        if turn.last_episode and ship.halite >= 500:
            converts(ship, turn)

            
    # Generate shipyard from the best ship
    option = 2

    # N of ships needed to create shipyard.
    create_shipyard = False

    if turn.num_shipyards == 1 and turn.num_ships > 8:
        create_shipyard = True

    elif turn.num_shipyards == 2 and turn.num_ships > 17:
        create_shipyard = True

    elif turn.num_shipyards == 3 and turn.num_ships > 20:
        create_shipyard = True

    elif turn.num_shipyards == 4 and turn.num_ships > 26:
        create_shipyard = True

    elif turn.num_shipyards == 5 and turn.num_ships > 30:
        create_shipyard = True

    else:
        create_shipyard = False
            
    # OPTION 1: Creative shipyards in defensive positions, close to other ally shipyards.
    if option == 1:
        if ((turn.num_shipyards < CONFIG_MAX_SHIPYARDS) and create_shipyard and board.step < 200) \
                or ((turn.num_shipyards < CONFIG_MAX_SHIPYARDS - 1) and create_shipyard and board.step < 300):
            
            max_halite_score = 0
            max_ship = None

            # We itarate over our ships to find the best one to convert.
            for ship in me.ships:
                halite_score = 0
                if ship.next_action: continue

                # Only consider ships that can create a shipyard and left us with a certain margin of halite
                # to be able to defend it.
                if ship.halite + turn.total_halite >= 1000:
                    if len(me.shipyards) == 1:
                        for sy in me.shipyards:
                            # Iterate over all cells to find the best one to convert from.
                            # We prioritize having the cell closer and with less halite.
                            # Also being surrounded by cells with high halite.
                            for pt, c in board.cells.items():
                                d = dist(ship.position, pt)
                                if d == 0: # Substract score if the position have much halite
                                    halite_score -= c.halite*3
                                # The cells around us are the ones that give the positive score
                                # since are the ones we will mine.
                                elif d < 3:
                                    halite_score += (c.halite / d)

                            d = dist(ship.position, sy.position)
                            if d == 0:  # shipyard here
                                halite_score -= 10000000
                            elif d < 8:
                                halite_score -= 30
                            if d < 6:
                                halite_score -= 10000000
                            if d > 10:
                                halite_score -= 10000000
                                
                    # When we need to create the third or above shipayard, we prioritize position
                    # not halite's surroundings.
                    elif len(me.shipyards) < 4 and board.cells[ship.position].halite == 0:
                        # We initialize with halite's cell
                        halite_score = - board.cells[ship.position].halite
                        for sy in me.shipyards:
                            d = dist(ship.position, sy.position)
                            if d == 0: # shipyard here
                                halite_score -= 10000000
                            if d < 6:
                                halite_score -= 10000000
                            if d >= 6 and d < 9:
                                # we create if cell has not more than 100 haite.
                                halite_score += 50
                            if d >= 9:
                                halite_score -= 75
                                    
                    elif len(me.shipyards) < 6 and board.cells[ship.position].halite == 0:
                        # We initialize with halite's cell
                        halite_score = - board.cells[ship.position].halite
                        for sy in me.shipyards:
                            d = dist(ship.position, sy.position)
                            if d == 0: # shipyard here
                                halite_score -= 10000000
                            if d < 6:
                                halite_score -= 10000000
                            if d >= 6 and d < 9:
                                # we create if cell has not more than 100 haite.
                                halite_score += 100
                            if d >= 9:
                                halite_score -= 50
                        
                # We select the ship with the maximum halite score. 
                # We always want it to be higher than 0 (value of initialization).
                # Also we only consider ships that are not in danger, oterwise they would convert and die.
                # Ie, we check that the position where we want to transform, is not in the avoidance matrix.
                positions_to_avoid = make_avoidance_matrix(ship.halite)               
                if halite_score > max_halite_score and not positions_to_avoid[ship.position.y, ship.position.x]:
                    max_halite_score = halite_score
                    max_ship = ship

            # If we have find a ship with max halite score more than 0, then convert it.
            # we convert 1 per turn max.
            if max_ship:
                converts(max_ship, turn)
        
    # OPTION 2: Create shipyards on positions with more halite, controlling the distance between shipyards.
    elif option == 2:
        # Generate shipyard from the best ship
        # Notice we create shipyards based on the num of ships and num of shipays we currently have.
        if ((turn.num_shipyards < CONFIG_MAX_SHIPYARDS) and create_shipyard and board.step < 200) \
                or ((turn.num_shipyards < CONFIG_MAX_SHIPYARDS - 1) and create_shipyard and board.step < 300):
            max_halite_score = 0
            max_ship = None

            # We itarate over our ships to find the best one to convert.
            for ship in me.ships:
                halite_score = 0
                if ship.next_action: continue

                # Only consider ships that can create a shipyard and left us with a certain margin of halite
                # to be able to defend it. Also they must be separated from enemy shipyards.
                d_nearest_shipyard = nearest_enemy_shipyard(board, ship.position)
                if ship.halite + turn.total_halite >= 1000 and d_nearest_shipyard >= 6:
                    if len(me.shipyards) == 1:
                        for sy in me.shipyards:
                            # Iterate over all cells to find the best one to convert from.
                            # We prioritize having the cell closer and with less halite.
                            # Also being surrounded by cells with high halite.
                            for pt, c in board.cells.items():
                                d = dist(ship.position, pt)
                                if d == 0: # Substract score if the position have much halite
                                    halite_score -= c.halite*3
                                # The cells around us are the ones that give the positive score
                                # since are the ones we will mine.
                                elif d < 3:
                                    halite_score += (c.halite / d)

                            d = dist(ship.position, sy.position)
                            if d == 0:  # shipyard here
                                halite_score -= 10000000
                            elif d < 8:
                                halite_score -= 30
                            if d < 6:
                                halite_score -= 10000000
                            if d > 8:
                                halite_score -= 10000000

                    elif board.cells[ship.position].halite == 0 and d_nearest_shipyard >= 8:
                        # We initialize with halite's cell
                        halite_score = 0
                        for sy in me.shipyards:
                            for pt, c in board.cells.items():
                                d = dist(ship.position, pt)
                                if d == 0: # Substract score if the position have much halite
                                    halite_score -= c.halite
                                else:
                                    halite_score += (c.halite / d)

                            d = dist(ship.position, sy.position)
                            if d == 0: # shipyard here
                                halite_score -= 10000000
                            if d < 6:
                                halite_score -= 10000000
                            else:
                                halite_score -= 30 / d 
                                
                    elif len(me.shipyards) >= 4 and board.cells[ship.position].halite == 0 and d_nearest_shipyard >= 5:
                        # We initialize with halite's cell
                        halite_score = 0
                        for sy in me.shipyards:
                            for pt, c in board.cells.items():
                                d = dist(ship.position, pt)
                                if d == 0: # Substract score if the position have much halite
                                    halite_score -= c.halite
                                else:
                                    halite_score += (c.halite / d)

                            d = dist(ship.position, sy.position)
                            if d == 0: # shipyard here
                                halite_score -= 10000000
                            if d < 6:
                                halite_score -= 10000000
                            else:
                                halite_score -= 30 / d 

                # We select the ship with the maximum halite score. 
                # We always want it to be higher than 0 (value of initialization).
                # Also we only consider ships that are not in danger, oterwise they would convert and die.
                # Ie, we check that the position where we want to transform, is not in the avoidance matrix.
                positions_to_avoid = make_avoidance_matrix(ship.halite)               
                if halite_score > max_halite_score and not positions_to_avoid[ship.position.y, ship.position.x]:
                    max_halite_score = halite_score
                    max_ship = ship

            # If we have find a ship with max halite score more than 0, then convert it.
            # we convert 1 per turn max.
            if max_ship:
                converts(max_ship, turn)

    
def ship_moves(board):
    # List of my ships that doesn't have an action assigned. 
    # For instance we may have already assigned the action convert to the ship.
    # Then we do not need to assign a move.
    ships = [ship for ship in me.ships if ship.next_action is None]
    
    player_more_ships, player_more_halite = compute_winning_position(board)
    
    # Assign targets to ships.
    assign_targets(board, ships, player_more_ships, player_more_halite)
    
    # This dictionary records the actions for each ship.
    actions = {}
    
    for ship in ships:
        # If the ship has a target, we compute the actions we need to do
        # to get the ship to that target.
        if ship.id in ship_target:
            a, delta = dirs_to(ship.position, ship_target[ship.id], size=size)
            actions[ship.id] = a
            
        # If the ship doesn't have a target, we give it a random movement
        else:
            actions[ship.id] = [random.choice(all_actions)]

    for ship in ships:
        action = 0
        x = ship.position
        
        # Generate matrix of places to attack and places to avoid
        avoid = make_avoidance_matrix(ship.halite)
        attack = make_attack_matrix(ship.halite)
        attack_shipyard = make_attack_shipyard_matrix(ship.halite)
        
        # action_list is the actions of the ship deduced from before (going to the target)
        # + None + the four movements (all actions).
        # Now that we have a list of actions for the next turn, see if we should priorize
        # some of them.
        # We shuffle otherwise we always prioritize first one diretion.
        random.shuffle(all_actions)
        action_list = actions[ship.id] + [None] + all_actions
        
        # We iterate over all the actions and see if some action can lead us to attack a shipyard.
        # If yes, then we attack.
        for a in all_actions:
            # We compute the new positon after the action.
            m = move(x, a)
            distance_nearest_shipyard, _ = nearest_shipyard(m)
            # We may select an atack here but if it later falls in an avoidance condition, the action
            # will be invalidated.
            if attack_shipyard[m.y, m.x]:
                # If my ship has != 0 halite, only attack if enemy cannot spawn a new ship.
                # Also we prioritize attack only if it is close to our shipyards.
                if (ship.halite == 0 or board.players[board.cells[m].shipyard.player_id].halite < 500) \
                        and distance_nearest_shipyard < 8:
                    print('ship id {} attacking {}'.format(ship.id, a))
                    # We put the action as the first one in order to priorize it.
                    action_list.insert(0, a)
                    break
        
        # We iterate over all the actions and see if some action can lead us to attack a ship.
        # If yes, then we attack.
        # Note: We iterate over the actions that we have previously deduced from going 
        # to the max C point + all other actions.
        # We put a condition on the attack in order to not follow the enemy's ship continuously
        # going super far away of my own shipyards.
    
        for a in all_actions:
            # We compute the new positon after the action.
            m = move(x, a)
            # We only attack if we don't separate too much from our shipyards.
            d2, shipyard_position = nearest_shipyard(m)
            if attack[m.y, m.x] and d2 < ATTACK_SEPARATION_SHPIPYARDS: 
                    # and board.cells[m].halite >= ATTACK_MIN_HALITE_CELL:
                print('ship id {} attacking {}'.format(ship.id, a))
                # We put the action as the first one in order to priorize it.
                action_list.insert(0, a)
                break
                
        # Notice we prioritize to attack ships before shipyards as long as they 
        # are closer than ATTACK_SEPARATION_SHPIPYARDS
        
        # We iterate again over all actions (removing duplicates).
        # If the action brings us to some cell that we should avoid, then don't do that action.
        action_list = remove_dups(action_list)
        for a in action_list:
            m = move(x, a)
            d2, shipyard_position = nearest_shipyard(m)
            if avoid[m.y, m.x] and (ship.halite != 0 or d2 > 1):
                print('ship id {} avoiding {}'.format(ship.id, a))
            # We select the first action in the list that is not in an avoidance cell.
            if m not in turn.taken and not (avoid[m.y, m.x] and (ship.halite != 0 or d2 > 1)):
                action = a
                break
                
        # CLOSING DOWN THE GAME
        # Find nearest shipyard and go there. 
        if board.step >= STEP_START_CLOSE_DOWN and ship.halite != 0 and len(me.shipyards) > 0:
            d2, shipyard_position = nearest_shipyard(ship.position)
            actions_closing_game, delta = dirs_to(ship.position, shipyard_position)
            action_list_closing = actions_closing_game + [None] + all_actions
            action_list_closing = remove_dups(action_list_closing)
            for a in action_list_closing:
                m = move(x, a)
                if not avoid[m.y, m.x]:
                    action = a
                    break
        # If all our possible actions are in avoidance positions, we end up with action = 0, which is
        # the initialization value.
        # In such case, we have two options: 
        # 1. we take the actions derived from the target score 
        # as long as it is not a stay action, ie None. 
        # This is because stay will probably get us killed.
        # 2. prioratize cells with 0 halite or moving to position where there was an enemy
        # (since he probably will move). They have more chances to survive.
        option = 2
        if action == 0:
            if option == 1:
                for a in actions[ship.id]:
                    m = move(x, a)
                    if a is not None and m not in turn.taken:
                        action = a
                        break
            elif option == 2:
                for a in all_actions:
                    m = move(x, a)
                    if a is not None and m not in turn.taken and board.cells[m].halite == 0:
                        action = a
                        break
                
                
            # Double check if we still have no action. 
            # If that is the case, we give a random movement, not none and
            # not interfering with any other ally ship movement.                
            count = 0
            while action == 0:
                a = random.choice(all_actions)
                m = move(x, a)
                if m not in turn.taken:
                    action = a
                    break
                
                count += 1
                if count > 4:
                    action = random.choice(all_actions)
                    break

        ship.next_action = action
        turn.taken[m] = 1


# Returns the commands we send to our ships and shipyards.
# It must be last function in the file.
def agent(obs, config):
    global size
    global start
    global prev_board
    global me
    global did_init
    
    # Do initialization for the first time.
    start_step = time.time()
    if start is None:
        start = time.time()
    if not did_init:
        init(obs, config)
        did_init = True
    board = Board(obs, config)
    me = board.current_player
    set_turn_data(board)
    print('\n################################################################')
    print('STEP {}'.format(board.step))
    print('n ships: {}. n shipyards: {}.'.format(turn.num_ships, turn.num_shipyards))
    # print_enemy_ships(board)
    # shipyard_defense_spawn_ships(board)
    ship_converts(board)
    ship_moves(board)
    shipyard_actions(board)
    shipyard_defense_move_ships(board)
    print_actions(board)
    # print('time this turn: {:8.3f} total elapsed {:8.3f}'.format(
    #       time.time() - start_step,
    #       time.time() - start))
    
    return me.next_actions



# Test Agent

In [ ]:
# Play against yourself without an ERROR or INVALID.
# Note: The first episode in the competition will run this to weed out erroneous agents.
# env.run(["/kaggle/working/submission.py", "/kaggle/working/submission.py"])
# print("EXCELLENT SUBMISSION!" if env.toJSON()["statuses"] == ["DONE", "DONE"] else "MAYBE BAD SUBMISSION?")

# Play as the first agent against default "shortest" agent.
#env.run(["/kaggle/working/submission.py"])

#env.render(mode="ipython", width=800, height=600)

In [ ]:
env.run(["/kaggle/working/submission.py",
         "../input/halitev66/submission (11).py",
         "../input/halitev67/submission (12).py",
         "../input/halitev64/submission (10).py",
        ])

In [ ]:
env.render(mode="ipython", width=800, height=600)

env.run(["/kaggle/working/submission.py",
         "../input/halitev12/submission.py",
         "../input/optimus-mine-agent/submission.py",
         "../input/halitev12/submission.py"])

env.render(mode="ipython", width=800, height=600)

env.run(["/kaggle/working/submission.py",
         "random",
         "random",
         "random"])

env.render(mode="ipython", width=800, height=600)

# Submit to Competition

1. Commit this kernel.
1. View the commited version.
1. Go to "Data" section and find submission.py file.
1. Click "Submit to Competition"
1. Go to My Submissions to view your score and episodes being played.